<a href="https://colab.research.google.com/github/sebasanjose/AI-Notebooks/blob/main/add_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!which nvcc

/usr/local/cuda/bin/nvcc


In [ ]:
!nvidia-smi

Mon Jun 24 12:22:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%writefile hello.cu

#include<stdio.h>
__global__ void hello(void)
{
    printf("GPU: Hello!\n");
}
int main(int argc,char **argv)
{
    printf("CPU: Hello!\n");
    hello<<<1,10>>>();
    cudaDeviceReset();
    return 0;
}

Writing hello.cu


In [ ]:
!nvcc hello.cu -o hello

In [ ]:
!./hello

CPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!


In [4]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add.cu


In [5]:
!nvcc add.cu -o add_cuda

In [6]:
!./add_cuda

Max error: 0


In [2]:
!ls -la /usr/local/cuda-12.2/bin

total 150620
drwxr-xr-x 3 root root     4096 Nov 10  2023 .
drwxr-xr-x 1 root root     4096 Nov 10  2023 ..
-rwxr-xr-x 1 root root    84752 Aug 16  2023 bin2c
-rwxr-xr-x 1 root root      112 Aug 16  2023 compute-sanitizer
drwxr-xr-x 2 root root     4096 Nov 10  2023 crt
-rwxr-xr-x 1 root root  6871800 Aug 16  2023 cudafe++
-rwxr-xr-x 1 root root 15645016 Aug 16  2023 cuda-gdb
-rwxr-xr-x 1 root root   807856 Aug 16  2023 cuda-gdbserver
-rwxr-xr-x 1 root root    75928 Aug 16  2023 cu++filt
-rwxr-xr-x 1 root root   527840 Aug 16  2023 cuobjdump
-rwxr-xr-x 1 root root   285728 Aug 16  2023 fatbinary
-rwxr-xr-x 1 root root     3826 Aug 23  2023 ncu
-rwxr-xr-x 1 root root     3616 Aug 23  2023 ncu-ui
-rwxr-xr-x 1 root root 21281160 Aug 16  2023 nvcc
-rwxr-xr-x 1 root root    10456 Aug 16  2023 __nvcc_device_query
-rw-r--r-- 1 root root      417 Aug 16  2023 nvcc.profile
-rwxr-xr-x 1 root root 50645896 Aug 16  2023 nvdisasm
-rwxr-xr-x 1 root root 26072056 Aug 16  2023 nvlink
-rwxr-xr-x 1 root

In [3]:
!ls

sample_data


In [7]:
!/usr/local/cuda-12.2/bin/nvprof ./add_cuda

==1218== NVPROF is profiling process 1218, command: ./add_cuda
Max error: 0
==1218== Profiling application: ./add_cuda
==1218== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  108.52ms         1  108.52ms  108.52ms  108.52ms  add(int, float*, float*)
      API calls:   57.35%  147.54ms         2  73.771ms  51.024us  147.49ms  cudaMallocManaged
                   42.18%  108.53ms         1  108.53ms  108.53ms  108.53ms  cudaDeviceSynchronize
                    0.25%  649.79us         2  324.90us  265.44us  384.35us  cudaFree
                    0.13%  329.07us         1  329.07us  329.07us  329.07us  cudaLaunchKernel
                    0.08%  198.50us       114  1.7410us     218ns  75.455us  cuDeviceGetAttribute
                    0.01%  13.222us         1  13.222us  13.222us  13.222us  cuDeviceGetName
                    0.00%  8.1930us         1  8.1930us  8.1930us  8.1930us  cuDeviceGetPCIBusId
        